In [1]:
import tensorflow as tf
import time
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np


In [5]:
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # When the data (tensor) is too small, let's give them a better error message
        if self.num_batches==0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)


    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

### Parameters
#### Batch, number_of_batch, batch_size and seq_length
what is batch, number_of_batch, batch_size and seq_length in the charcter level example?  

Lets assume the input is this sentence: '__here is an example__'. Then:
- txt_length = 18  
- seq_length = 3  
- batch_size = 2  
- number_of_batchs = 18/3*2 = 3
- batch = array (['h','e','r'],['e',' ','i'])
- sample Seq = 'her'  

In [2]:
seq_length = 50 # RNN sequence length
batch_size = 60  # minibatch size, i.e. size of data in each epoch
num_epochs = 125 # you should change it to 50 if you want to see a relatively good results
learning_rate = 0.002
decay_rate = 0.97
rnn_size = 128 # size of RNN hidden state (output dimension)
num_layers = 2 #number of layers in the RNN

In [3]:
!wget -nv -O input.txt https://ibm.box.com/shared/static/a3f9e9mbpup09toq35ut7ke3l3lf03hg.txt 
with open('input.txt', 'r') as f:
    read_data = f.read()
    print (read_data[0:100])
f.closed

2019-12-13 17:08:15 URL:https://public.boxcloud.com/d/1/b1!0SWDfH0nHO7c_Hy_GAijVJLLNHIet7oVbSGl6fzoVOIFwWaiN7jJjpuFbvi8ADIgXVkb4DtpnKgvIHGm-5qVapzgSh3P08E0o4X9XQkNL9BW-phBQHpqqqshqGiGF-Mgi1S6KXhtSHi_iBpnU131TP-iLK98HbGNRAmrO7h5nVZrDPUmhl7tYA0Ou39W1jopuEYFUTU0sjnxUfnAUTdPCxjCN8alovTRyVS_52VW3ZgBtzs0rD-sxa6TI8DiDZpSaMN-5bL0yIqAhZGOuJTF5huZiCN4U4V2xzpxqFjUGrVYacysVgEjapk_AOOwl7jzDA3643uoMNpV1jzrN_KLDzH16QQg84AMVptOibF1wfXwcatwq8MV6YCI7SdEX3blQQfg0oopRM4vYB6f9TQWJTidWSWn-Ms_uv1tHSHpnYxS0Z3GokWfQ7udkXdAfNBDDaHIHWs45ni9SlHX-w1eeMPu_qF3LPC6a9rZDgGQOIBxBYVA_iPhzJmF8HoqMxI2afc1ZpfvFyhOOopk2U-Rsq_T3m-iC0TDiA9ctbCtYEBlwijftcE5H8w3l3vdYP9153smSab3bn7QsFx1-DwhVwSh9uPaLxqW8xzXzHB26yVc99shLMy0lZ2UILUQVbQIwj_-XlWv10wCXCXNvnpxNpPKb6rQ8ss5dTcwcPqPJ5gyEzjcan9vv5Oxy3Yw1lJzrPiu9ohgmHL_MDwSZRJs733nsw7noe21PtuL72NUZ33ViSiUjk_6N7ajOraffZkN9COghfrV4sKL3965Mi5TN8O13dT9_VEbjQKSDsIgH0iygCcT2PVZ6Ryakkim0TB51FuiAdX3yS1psLBCwO5cWoggq7koII2CMuQkvUKfdfCki14Kd4I7vIZIe6ZnB9LDnTGiBLtSFP-pRNyKKU6oHTkgsbDDct62JmPa9ay-yfh4W

True

In [6]:
data_loader = TextLoader('', batch_size, seq_length)
vocab_size = data_loader.vocab_size
print ("vocabulary size:" ,data_loader.vocab_size)
print ("Characters:" ,data_loader.chars)
print ("vocab number of 'F':",data_loader.vocab['F'])
print ("Character sequences (first batch):", data_loader.x_batches[0])

reading text file
vocabulary size: 65
Characters: (' ', 'e', 't', 'o', 'a', 'h', 's', 'r', 'n', 'i', '\n', 'l', 'd', 'u', 'm', 'y', ',', 'w', 'f', 'c', 'g', 'I', 'b', 'p', ':', '.', 'A', 'v', 'k', 'T', "'", 'E', 'O', 'N', 'R', 'S', 'L', 'C', ';', 'W', 'U', 'H', 'M', 'B', '?', 'G', '!', 'D', '-', 'F', 'Y', 'P', 'K', 'V', 'j', 'q', 'x', 'z', 'J', 'Q', 'Z', 'X', '3', '&', '$')
vocab number of 'F': 49
Character sequences (first batch): [[49  9  7 ...  1  4  7]
 [19  4 14 ... 14  9 20]
 [ 8 20 10 ...  8 10 18]
 ...
 [21  2  0 ...  0 21  0]
 [ 9  7  7 ...  0  2  3]
 [ 3  7  0 ...  5  9 23]]


In [7]:
x,y = data_loader.next_batch()
x

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ...,
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]])

In [8]:
x.shape  #batch_size =60, seq_length=50

(60, 50)

Here, __y__ is the next character for each character in __x__:

In [9]:
y

array([[ 9,  7,  6, ...,  4,  7,  0],
       [ 4, 14, 22, ...,  9, 20,  5],
       [20, 10, 29, ..., 10, 18,  4],
       ...,
       [ 2,  0,  6, ..., 21,  0,  6],
       [ 7,  7,  4, ...,  2,  3,  0],
       [ 7,  0, 33, ...,  9, 23,  0]])

### LSTM Architecture
Each LSTM cell has 5 parts:
1. Input
2. prv_state
3. prv_output
4. new_state
5. new_output


- Each LSTM cell has an input layre, which its size is 128 units in our case. The input vector's dimension also is 128, which is the dimensionality of embedding vector, so called, dimension size of W2V/embedding, for each character/word.
- Each LSTM cell has a hidden layer, where there are some hidden units. The argument n_hidden=128 of BasicLSTMCell is the number of hidden units of the LSTM (inside A). It keeps the size of the output and state vector. It is also known as, rnn_size, num_units, num_hidden_units, and LSTM size
- An LSTM keeps two pieces of information as it propagates through time: 
    - __hidden state__ vector: Each LSTM cell accept a vector, called __hidden state__ vector, of size n_hidden=128, and its value is returned to the LSTM cell in the next step. The __hidden state__ vector; which is the memory of the LSTM, accumulates using its (forget, input, and output) gates through time. "num_units" is equivalant to "size of RNN hidden state". number of hidden units is the dimensianality of the output (= dimesianality of the state) of the LSTM cell.
    - __previous time-step output__: For each LSTM cell that we initialize, we need to supply a value (128 in this case) for the hidden dimension, or as some people like to call it, the number of units in the LSTM cell. 


#### num_layers = 2 
- number of layers in the RNN, is defined by num_layers
- An input of MultiRNNCell is __cells__ which is list of RNNCells that will be composed in this order.

In [10]:
cell = tf.contrib.rnn.BasicRNNCell(rnn_size)

In [11]:
# a two layer cell
stacked_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)

In [12]:
# hidden state size
stacked_cell.output_size

128

__state__ varibale keeps output and new_state of the LSTM, so it is a touple of size:

In [13]:
stacked_cell.state_size

(128, 128)

Lets define input data:

In [14]:
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])# a 60x50
input_data

<tf.Tensor 'Placeholder:0' shape=(60, 50) dtype=int32>

and target data:

In [15]:
targets = tf.placeholder(tf.int32, [batch_size, seq_length]) # a 60x50
targets

<tf.Tensor 'Placeholder_1:0' shape=(60, 50) dtype=int32>

The memory state of the network is initialized with a vector of zeros and gets updated after reading each character.

__BasicRNNCell.zero_state(batch_size, dtype)__ Return zero-filled state tensor(s). In this function, batch_size
representing the batch size.

In [16]:
initial_state = stacked_cell.zero_state(batch_size, tf.float32) #why batch_size ? 60x128

Lets check the value of the input_data again:

In [17]:
session = tf.Session()
feed_dict={input_data:x, targets:y}
session.run(input_data, feed_dict)

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ...,
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]], dtype=int32)

In [18]:
with tf.variable_scope('rnnlm', reuse=False):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
    softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65)
    #with tf.device("/cpu:0"):
        
    # embedding variable is initialized randomely
    embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128

    # embedding_lookup goes to each row of input_data, and for each character in the row, finds the correspond vector in embedding
    # it creates a 60*50*[1*128] matrix
    # so, the first elemnt of em, is a matrix of 50x128, which each row of it is vector representing that character
    em = tf.nn.embedding_lookup(embedding, input_data) # em is 60x50x[1*128]
    # split: Splits a tensor into sub tensors.
    # syntax:  tf.split(split_dim, num_split, value, name='split')
    # it will split the 60x50x[1x128] matrix into 50 matrix of 60x[1*128]
    inputs = tf.split(em, seq_length, 1)
    # It will convert the list to 50 matrix of [60x128]
    inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

In [19]:
session.run(tf.global_variables_initializer())
#print embedding.shape
session.run(embedding)

array([[-0.17625183,  0.13357027,  0.00036031, ..., -0.02504167,
         0.1532857 ,  0.01203077],
       [-0.13017616,  0.15892933,  0.07853533, ...,  0.02669726,
         0.13156472,  0.11107837],
       [-0.14612716, -0.04875024,  0.04737343, ..., -0.150432  ,
        -0.12615632, -0.12329477],
       ...,
       [ 0.10898928, -0.08136004, -0.13546182, ...,  0.05370633,
         0.07723577, -0.17375073],
       [ 0.15424486,  0.07611503,  0.1593271 , ..., -0.13438112,
        -0.01541285, -0.10879839],
       [-0.14512688, -0.14794531,  0.02864063, ...,  0.0290288 ,
        -0.11088454, -0.01355083]], dtype=float32)

In [20]:
em = tf.nn.embedding_lookup(embedding, input_data)
emp = session.run(em,feed_dict={input_data:x})
print (emp.shape)
emp[0]

(60, 50, 128)


array([[-0.04376492, -0.16119222,  0.10852246, ...,  0.14592357,
         0.07295316, -0.06529608],
       [-0.04553311, -0.1111398 , -0.03309092, ...,  0.07616119,
         0.0100109 , -0.14729097],
       [ 0.1751888 , -0.08107515, -0.00191666, ..., -0.03158636,
         0.16963197, -0.14585912],
       ...,
       [-0.13017616,  0.15892933,  0.07853533, ...,  0.02669726,
         0.13156472,  0.11107837],
       [ 0.05240321,  0.1522957 , -0.06164466, ..., -0.08193675,
         0.09884982,  0.16868593],
       [ 0.1751888 , -0.08107515, -0.00191666, ..., -0.03158636,
         0.16963197, -0.14585912]], dtype=float32)

Let's consider each sequence as a sentence of length 50 characters, then, the first item in __inputs__ is a [60x128] vector which represents the first characters of 60 sentences.

In [21]:
inputs = tf.split(em, seq_length, 1)
inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
inputs[0:5]

[<tf.Tensor 'Squeeze:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_2:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_4:0' shape=(60, 128) dtype=float32>]

### Feeding a batch of 50 sequence to a RNN:

The feeding process for iputs is as following:

- Step 1:  first character of each of the 50 sentences (in a batch) is entered in parallel.  
- Step 2:  second character of each of the 50 sentences is input in parallel. 
- Step n: nth character of each of the 50 sentences is input in parallel.  

The parallelism is only for efficiency.  Each character in a batch is handled in parallel,  but the network sees one character of a sequence at a time and does the computations accordingly. All the computations involving the characters of all sequences in a batch at a given time step are done in parallel. 

In [22]:
session.run(inputs[0],feed_dict={input_data:x})

array([[-0.04376492, -0.16119222,  0.10852246, ...,  0.14592357,
         0.07295316, -0.06529608],
       [-0.08977027, -0.02975172,  0.15910296, ..., -0.0050181 ,
        -0.01723287,  0.07445626],
       [ 0.11965005, -0.08448334,  0.1368093 , ..., -0.06466489,
         0.15687434,  0.098013  ],
       ...,
       [ 0.05284545, -0.05131595, -0.11537419, ..., -0.12069497,
         0.05327705,  0.08198164],
       [-0.04553311, -0.1111398 , -0.03309092, ...,  0.07616119,
         0.0100109 , -0.14729097],
       [-0.15745601, -0.10621195, -0.01758568, ...,  0.15801616,
         0.10293601,  0.03540783]], dtype=float32)

Feeding the RNN with one batch, we can check the new output and new state of network:

In [23]:
#outputs is 50x[60*128]
outputs, new_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, stacked_cell, loop_function=None, scope='rnnlm')
new_state

(<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_98:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_99:0' shape=(60, 128) dtype=float32>)

In [24]:
outputs[0:5]

[<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_5:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_7:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_9:0' shape=(60, 128) dtype=float32>]

Let's check the output of network after feeding it with first batch:

In [25]:
first_output = outputs[0]
session.run(tf.global_variables_initializer())
session.run(first_output,feed_dict={input_data:x})

array([[-4.22630310e-02,  3.42363976e-02, -4.26930152e-02, ...,
        -7.63965771e-02,  5.08053042e-02, -2.53888462e-02],
       [ 8.50468725e-02,  6.65854570e-03,  9.48300287e-02, ...,
         1.05712995e-01,  4.03932147e-02,  8.53596181e-02],
       [ 7.97982365e-02, -4.52836752e-02,  6.63619041e-02, ...,
        -5.76792993e-02, -3.19513753e-02, -8.55729878e-02],
       ...,
       [ 1.75490262e-04,  1.54609039e-01,  6.71157464e-02, ...,
        -2.39158615e-01, -9.43184178e-03, -5.52322567e-02],
       [ 1.15086639e-03,  2.39509586e-02,  1.65911876e-02, ...,
         6.20116033e-02, -1.07426845e-01, -4.55905534e-02],
       [-4.69695441e-02,  6.68449029e-02, -3.09155472e-02, ...,
        -6.50945082e-02, -1.05036527e-01, -7.92484209e-02]], dtype=float32)

As it was explained, __outputs__ variable is a 50x[60x128] tensor. We need to reshape it back to [60x50x128] to be able to calculate the probablity of the next character using the softmax. The __softmax_w__ shape is [rnn_size, vocab_size],whihc is [128x65] in our case. Threfore, we have a fully connected layer on top of LSTM cells, which help us to decode the next charachter. We can use the __softmax(output * softmax_w + softmax_b)__ for this purpose. The shape of the matrixis would be:

softmax([60x50x128]x[128x65]+[1x65]) = [60x50x65]

In [26]:
output = tf.reshape(tf.concat( outputs,1), [-1, rnn_size])
output

<tf.Tensor 'Reshape:0' shape=(3000, 128) dtype=float32>

In [27]:
logits = tf.matmul(output, softmax_w) + softmax_b
logits

<tf.Tensor 'add:0' shape=(3000, 65) dtype=float32>

In [28]:
probs = tf.nn.softmax(logits)
probs

<tf.Tensor 'Softmax:0' shape=(3000, 65) dtype=float32>

Here is the probablity of the next chracter in all batches:

In [29]:
session.run(tf.global_variables_initializer())
session.run(probs,feed_dict={input_data:x})

array([[0.01446933, 0.01428786, 0.01201231, ..., 0.01596677, 0.01422769,
        0.01695991],
       [0.01350139, 0.01475812, 0.0099122 , ..., 0.01489287, 0.01394811,
        0.01255622],
       [0.01115899, 0.013269  , 0.01354345, ..., 0.01887721, 0.01565605,
        0.01842083],
       ...,
       [0.01135813, 0.01407566, 0.01721771, ..., 0.02178912, 0.01587244,
        0.01739761],
       [0.01347092, 0.0182619 , 0.01448074, ..., 0.01799878, 0.02222749,
        0.01339769],
       [0.0125967 , 0.01077188, 0.01329833, ..., 0.01558688, 0.01449924,
        0.01622051]], dtype=float32)

In [30]:
grad_clip =5.
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'rnnlm/softmax_w:0' shape=(128, 65) dtype=float32_ref>,
 <tf.Variable 'rnnlm/softmax_b:0' shape=(65,) dtype=float32_ref>,
 <tf.Variable 'rnnlm/embedding:0' shape=(65, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(128,) dtype=float32_ref>]

In [31]:
class LSTMModel():
    def __init__(self,sample=False):
        rnn_size = 128 # size of RNN hidden state vector
        batch_size = 60 # minibatch size, i.e. size of dataset in each epoch
        seq_length = 50 # RNN sequence length
        num_layers = 2 # number of layers in the RNN
        vocab_size = 65
        grad_clip = 5.
        if sample:
            print(">> sample mode:")
            batch_size = 1
            seq_length = 1
        # The core of the model consists of an LSTM cell that processes one char at a time and computes probabilities of the possible continuations of the char. 
        basic_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
        # model.cell.state_size is (128, 128)
        self.stacked_cell = tf.contrib.rnn.MultiRNNCell([basic_cell] * num_layers)

        self.input_data = tf.placeholder(tf.int32, [batch_size, seq_length], name="input_data")
        self.targets = tf.placeholder(tf.int32, [batch_size, seq_length], name="targets")
        # Initial state of the LSTM memory.
        # The memory state of the network is initialized with a vector of zeros and gets updated after reading each char. 
        self.initial_state = stacked_cell.zero_state(batch_size, tf.float32) #why batch_size

        with tf.variable_scope('rnnlm_class1'):
            softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
            softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65
            with tf.device("/cpu:0"):
                embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128
                inputs = tf.split(tf.nn.embedding_lookup(embedding, self.input_data), seq_length, 1)
                inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
                #inputs = tf.split(em, seq_length, 1)
                
                


        # The value of state is updated after processing each batch of chars.
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, self.initial_state, self.stacked_cell, loop_function=None, scope='rnnlm_class1')
        output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([self.logits],
                [tf.reshape(self.targets, [-1])],
                [tf.ones([batch_size * seq_length])],
                vocab_size)
        self.cost = tf.reduce_sum(loss) / batch_size / seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),grad_clip)
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    
    def sample(self, sess, chars, vocab, num=200, prime='The ', sampling_type=1):
        state = sess.run(self.stacked_cell.zero_state(1, tf.float32))
        #print state
        for char in prime[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return(int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [probs, state] = sess.run([self.probs, self.final_state], feed)
            p = probs[0]

            if sampling_type == 0:
                sample = np.argmax(p)
            elif sampling_type == 2:
                if char == ' ':
                    sample = weighted_pick(p)
                else:
                    sample = np.argmax(p)
            else: # sampling_type == 1 default:
                sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret

In [32]:
with tf.variable_scope("rnn"):
    model = LSTMModel()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs): # num_epochs is 5 for test, but should be higher
        sess.run(tf.assign(model.lr, learning_rate * (decay_rate ** e)))
        data_loader.reset_batch_pointer()
        state = sess.run(model.initial_state) # (2x[60x128])
        for b in range(data_loader.num_batches): #for each batch
            start = time.time()
            x, y = data_loader.next_batch()
            feed = {model.input_data: x, model.targets: y, model.initial_state:state}
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
        print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
                .format(e * data_loader.num_batches + b, num_epochs * data_loader.num_batches, e, train_loss, end - start))
        with tf.variable_scope("rnn", reuse=True):
            sample_model = LSTMModel(sample=True)
            print (sample_model.sample(sess, data_loader.chars , data_loader.vocab, num=50, prime='The ', sampling_type=1))
            print ('----------------------------------')

370/46375 (epoch 0), train_loss = 1.918, time/batch = 0.059
>> sample mode:
The bily with theal, by andone, fare thume onke't have
----------------------------------
741/46375 (epoch 1), train_loss = 1.749, time/batch = 0.060
>> sample mode:
The stost a itsel

Lire, to hepe:
Edwell proces Do see
----------------------------------
1112/46375 (epoch 2), train_loss = 1.669, time/batch = 0.063
>> sample mode:
The please surge of thy lial;
And thy raund an some of
----------------------------------
1483/46375 (epoch 3), train_loss = 1.625, time/batch = 0.061
>> sample mode:
The commondmence?

PONT:
That shall so ewer prove myso
----------------------------------
